### Importing the necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf

### Exploring the dataset

In [2]:
print(os.listdir('/kaggle/input/fer2013'))

In [3]:
train_path = '/kaggle/input/fer2013/train'
val_path = '/kaggle/input/fer2013/test'

In [4]:
# Importing TensorFlow libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import Sequential
from tensorflow import keras
from sklearn.model_selection import KFold

## Data Preprocessing
 

In [5]:
emotion_labels = sorted(os.listdir(train_path))
print(emotion_labels)

In [6]:
batch_size = 64
target_size = (48,48)

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=target_size,
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=True)

val_generator = val_datagen.flow_from_directory(
        val_path,
        target_size=target_size,
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

# Data Summarization

### Dimensions of the Dataset

In [7]:
input_shape = (48,48,1) # img_rows, img_colums, color_channels
num_classes = 7

### Statistical Summary of all attributes

In [8]:
print(os.listdir('/kaggle/input/fer2013/train'))

### Breakdown of all data class variable

In [9]:
def plot_images(img_dir, top=10):
    all_img_dirs = os.listdir(img_dir)
    img_files = [os.path.join(img_dir, file) for file in all_img_dirs][:5]
  
    plt.figure(figsize=(12, 12))
  
    for idx, img_path in enumerate(img_files):
        plt.subplot(5, 5, idx+1)
        img = plt.imread(img_path)
        plt.tight_layout()         
        plt.imshow(img, cmap='gray')

In [10]:
print('Angry: ')
print()
plot_images(train_path+'/angry')

In [11]:
print('Disgust: ')
print()
plot_images(train_path+'/disgust')

In [12]:
print('Fear: ')
print()
plot_images(train_path+'/fear')

In [13]:
print('Happy: ')
print()
plot_images(train_path+'/happy')

In [14]:
print('Neutral: ')
print()
plot_images(train_path+'/neutral')

In [15]:
print('Sad: ')
print()
plot_images(train_path+'/sad')

In [16]:
print('Surprise: ')
print()
plot_images(train_path+'/surprise')

## Data Visualization

In [17]:
emotions = os.listdir('/kaggle/input/fer2013/train')
for emotion in emotions:
    count = len(os.listdir(f'/kaggle/input/fer2013/train/{emotion}'))
    print(f'{emotion} faces={count}')

In [18]:
emotions = os.listdir('/kaggle/input/fer2013/test')
for emotion in emotions:
    count = len(os.listdir(f'/kaggle/input/fer2013/test/{emotion}'))
    print(f'{emotion} faces={count}')

In [19]:
emotions = os.listdir('/kaggle/input/fer2013/train')
values = [len(os.listdir(f'/kaggle/input/fer2013/train/{emotion}')) for emotion in emotions]
fig = plt.figure(figsize = (10, 5))

# creating the bar plot
plt.bar(emotions, values, color ='grey',
        width = 0.4)

plt.xlabel("Emotions")
plt.ylabel("No. of images")
plt.title("Train dataset overview")
plt.show()

In [20]:
emotions = os.listdir('/kaggle/input/fer2013/test')
values = [len(os.listdir(f'/kaggle/input/fer2013/test/{emotion}')) for emotion in emotions]
fig = plt.figure(figsize = (10, 5))

# creating the bar plot
plt.bar(emotions, values, color ='grey',
        width = 0.4)

plt.xlabel("Emotions")
plt.ylabel("No. of images")
plt.title("Test dataset overview")
plt.show()

## Python Packages

1. Numpy
2. Pandas
3. Matplotlib
4. OS
5. Tensorflow
6. Sklearn

# Supervised Learning

## 1. Convolutional Neural Network (CNN)

### Building the Model

In [21]:
model = Sequential()
model.add(layers.Conv2D(16,(5,5),padding='valid',input_shape = input_shape)) # Convolutional layers
model.add(layers.Activation('relu')) # activation functions
model.add(layers.MaxPooling2D(pool_size=(2,2),strides=2,padding = 'valid')) # to reduce size of image
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(32,(5,5),padding='valid'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2),strides=2,padding = 'valid'))
model.add(layers.Dropout(0.6))

model.add(layers.Conv2D(64,(5,5),padding='valid'))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.8))
model.add(layers.Flatten())
model.add(layers.Dense(7)) # classification

model.add(layers.Activation('softmax'))

model.summary()

In [22]:
# Compile Model
optimizer = keras.optimizers.RMSprop(lr = 0.0001, decay = 1e-6)
model.compile(loss = 'binary_crossentropy',optimizer = optimizer, metrics = ['accuracy',keras.metrics.Precision(), keras.metrics.Recall()])

### Training the Model

In [23]:
num_epochs = 300

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VAL   = val_generator.n//val_generator.batch_size

In [ ]:
# Train Model
history = model.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, epochs=num_epochs, batch_size=batch_size, validation_data=val_generator, validation_steps=STEP_SIZE_VAL)
history

### Save Model

In [ ]:
models.save_model(model, 'CNN.h5')

### Evaluate Model

In [ ]:
cnn_score = model.evaluate_generator(val_generator, steps=STEP_SIZE_VAL) 
print('Test loss: ', cnn_score[0])
print('Test accuracy: ', cnn_score[1])

### Show Training History

In [ ]:
keys=history.history.keys()
print(keys)

def show_train_history(hisData,train,test): 
    plt.plot(hisData.history[train])
    plt.plot(hisData.history[test])
    plt.title('Training History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

show_train_history(history, 'loss', 'val_loss')
show_train_history(history, 'accuracy', 'val_accuracy')

## 2. Support Vector Machine (SVM)

### Building the Model

In [ ]:
num_epochs = 200
number_of_classes = 7

In [ ]:
model = Sequential()
model.add(layers.Conv2D(filters = 32, padding = "same",activation = "relu",kernel_size=3, strides = 2,input_shape=input_shape))
model.add(layers.MaxPool2D(pool_size=(2,2),strides = 2))

model.add(layers.Conv2D(filters = 32, padding = "same",activation = "relu",kernel_size=3))
model.add(layers.MaxPool2D(pool_size=(2,2),strides = 2))

model.add(layers.Flatten())
model.add(layers.Dense(128,activation="relu"))

#Output layer
model.add(layers.Dense(1,kernel_regularizer=regularizers.l2(0.01),activation = "linear"))

In [ ]:
model.add(layers.Dense(number_of_classes,kernel_regularizer = regularizers.l2(0.01),activation= "softmax"))
model.compile(optimizer="adam",loss="squared_hinge", metrics = ['accuracy'])
model.summary()

### Training the Model

In [ ]:
history = model.fit(x=train_generator, steps_per_epoch=STEP_SIZE_TRAIN, epochs=num_epochs, batch_size=batch_size, validation_data=val_generator, validation_steps=STEP_SIZE_VAL)

### Save Model

In [ ]:
models.save_model(model, 'SVM.h5')

### Evaluate Model

In [ ]:
svm_score = model.evaluate_generator(val_generator, steps=STEP_SIZE_VAL) 
print('Test loss: ', svm_score[0])
print('Test accuracy: ', svm_score[1])

### Show Training History

In [ ]:
keys=history.history.keys()
print(keys)

def show_train_history(hisData,train,test): 
    plt.plot(hisData.history[train])
    plt.plot(hisData.history[test])
    plt.title('Training History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

show_train_history(history, 'loss', 'val_loss')
show_train_history(history, 'accuracy', 'val_accuracy')

## 3. Artificial Neural Networks (ANN)

 

### Building the Model

In [ ]:
model_ann = Sequential()
model_ann.add(layers.Dense(16, input_shape=input_shape, activation='relu'))
model_ann.add(layers.Dropout(0.4))
model_ann.add(layers.Dense(32, activation='relu'))
model_ann.add(layers.Dropout(0.6))
model_ann.add(layers.Flatten())
model_ann.add(layers.Dense(7, activation='softmax'))

model_ann.summary()

In [ ]:
model_ann.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model_ann.fit(x=train_generator, steps_per_epoch=STEP_SIZE_TRAIN, epochs=num_epochs, batch_size=batch_size, validation_data=val_generator, validation_steps=STEP_SIZE_VAL)
history

### Save Model

In [ ]:
models.save_model(model, 'ANN.h5')

### Evaluate Model

In [ ]:
ann_score = model.evaluate_generator(val_generator, steps=STEP_SIZE_VAL) 
print('Test loss: ', ann_score[0])
print('Test accuracy: ', ann_score[1])

### Show Training History

In [ ]:
keys=history.history.keys()
print(keys)

def show_train_history(hisData,train,test): 
    plt.plot(hisData.history[train])
    plt.plot(hisData.history[test])
    plt.title('Training History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

show_train_history(history, 'loss', 'val_loss')
show_train_history(history, 'accuracy', 'val_accuracy')

## Comparing accuracy to find best Model

In [ ]:
# Accuracy of the different models
names = ['CNN', 'SVM', 'ANN']
compare_acc = [0.51, 0.7224, 0.4436]
# compare_acc = [0.94, 0.23, 0.45]
plt.figure(figsize=(20,6))


plt.barh(names,compare_acc, color=['#FFAEBC', '#FBE7C6', '#B4F8C8'])
for index, value in enumerate(compare_acc):
    plt.text(value, index, str(value))
plt.title('Comparing Accuracy of the Models')
plt.ylabel('Accuracy')
plt.xlabel('Models')

plt.show()